In [1]:
to_plot = False

In [2]:
#Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import copy

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.metrics import Metric

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [3]:
#Custom Imports
sys.path.append('../src')
import ml_functions as ml
import neural_network as nn

In [4]:
#Loading Data
df = pd.read_csv('../data/training_set_(50_50).csv', index_col = 0)

In [5]:
#Splitting Data
df_list_list, index_list_list = ml.split_data(df, 'ponzi', StratifiedKFold, n_splits= 10, shuffle = True, random_state = 2022)
data_list_list = copy.deepcopy(df_list_list)

In [6]:
#Scaling Data
col_ss = ['account_creation_time', 'gini_coefficient_accounts_received', 'gini_coefficient_accounts_sent', 'gini_coefficient_values_received', 'gini_coefficient_values_sent']
col_mm = []
col_pt = [x for x in df.columns if x not in col_ss and x != 'ponzi' and x not in col_mm]
scale_cols = [col_pt, col_ss, col_mm]
scale_methods = [PowerTransformer(method = 'box-cox'), StandardScaler(), MinMaxScaler()]

scalers = []
for i, (x_train, x_test, y_train, y_test) in enumerate(data_list_list):
    x_train, scaler = ml.scaling(x_train, scale_cols, scale_methods, default_scaler = 2)
    x_test, _ = ml.scaling(x_test, scale_cols, scale_methods, default_scaler = 2)
    
    x_train = tf.convert_to_tensor(x_train, dtype = tf.float64)
    x_test = tf.convert_to_tensor(x_test, dtype = tf.float64)
    y_train = tf.convert_to_tensor(y_train, dtype = tf.float64)
    y_test = tf.convert_to_tensor(y_test, dtype = tf.float64)
    
    data_list_list[i] = [x_train, x_test, y_train, y_test]
    scalers.append(scaler)

In [7]:
#Plotting Scaled Data
if to_plot:
    count = 0
    fig, ax = plt.subplots(nrows = len(data_list_list), ncols = 2, figsize = (12,32))
    for x_train, x_test, y_train, y_test in data_list_list:
        ax[count, 0].boxplot(x_train.numpy(), showfliers = False)
        ax[count, 0].set_title('train')
        ax[count, 1].boxplot(x_test.numpy(), showfliers = False)
        ax[count, 1].set_title('test')
        count += 1
    plt.show()

In [8]:
#Specifying NN Hyperparameters
#Design
layer_types = ['dropout', 'dense', 'dense', 'dense', 'batch_norm', 'dense', 'dense', 'dense', 'batch_norm', 'dropout']

total_n_nodes = 32*6
n_nodes_per_layer = total_n_nodes/sum(x == 'dense' for x in layer_types)

n_nodes = [n_nodes_per_layer if x == 'dense' else None for x in layer_types]
activations = [tf.nn.leaky_relu if x == 'dense' else None for x in layer_types]
dropout_param = 0.3
optimiser = tf.keras.optimizers.Adam
loss_fn = 'BinaryCrossentropy'
start_learn_rate = 1E-4


callback_es = tf.keras.callbacks.EarlyStopping(monitor = 'val_binary_accuracy', patience = 200, verbose = 1)
callback_rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 20,
                                                    min_delta = 1E-4, cooldown = 20, min_lr = 1E-7, verbose = 1)

#Training
n_epochs = 1000
batch_size = 32

In [9]:
#Creating NNs
nn_list = []
for x_train, x_test, y_train, y_test in df_list_list:
    nn_list.append(nn.NeuralNetwork(x_train, y_train, x_test, y_test,
                                    layer_types, n_nodes, activations, dropout_param, optimiser, loss_fn, start_learn_rate,
                                    filepath = r'C:\Users\quekh\Desktop\temp'))

In [10]:
#Training and saving NNs
result = []
for nn_m in nn_list:
    result.append(nn_m.model_fit(batch_size, n_epochs, callback_list = [callback_es, callback_rlr], save_checkpoints = True))
    nn_m.save_model()

Epoch 1/1000
1144/1172 [============================>.] - ETA: 0s - loss: 0.5628 - binary_accuracy: 0.7390 - precision: 0.7877 - recall: 0.6546
Epoch 00001: val_binary_accuracy improved from -inf to 0.49988, saving model to C:\Users\quekh\Desktop\temp\f1_do3.0_32lr_32lr_32lr_bn_32lr_32lr_32lr_bn_do3.0_Adam_1e04\ckpt\weights.01-0.70
1172/1172 [==============================] - 3s 2ms/step - loss: 0.5608 - binary_accuracy: 0.7396 - precision: 0.7871 - recall: 0.6569 - val_loss: 0.7014 - val_binary_accuracy: 0.4999 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/1000
1137/1172 [============================>.] - ETA: 0s - loss: 0.4458 - binary_accuracy: 0.7878 - precision: 0.8248 - recall: 0.7305
Epoch 00002: val_binary_accuracy improved from 0.49988 to 0.50036, saving model to C:\Users\quekh\Desktop\temp\f1_do3.0_32lr_32lr_32lr_bn_32lr_32lr_32lr_bn_do3.0_Adam_1e04\ckpt\weights.02-0.73
1172/1172 [==============================] - 2s 2ms/step - loss: 0.4450 - binary_accuracy: 0

1169/1172 [============================>.] - ETA: 0s - loss: 0.3185 - binary_accuracy: 0.8386 - precision: 0.8553 - recall: 0.8151
Epoch 00036: val_binary_accuracy did not improve from 0.78354
1172/1172 [==============================] - 2s 2ms/step - loss: 0.3185 - binary_accuracy: 0.8386 - precision: 0.8554 - recall: 0.8151 - val_loss: 1.0345 - val_binary_accuracy: 0.5004 - val_precision: 1.0000 - val_recall: 9.5969e-04
Epoch 37/1000
1150/1172 [============================>.] - ETA: 0s - loss: 0.3193 - binary_accuracy: 0.8405 - precision: 0.8554 - recall: 0.8193
Epoch 00037: ReduceLROnPlateau reducing learning rate to 1e-06.

Epoch 00037: val_binary_accuracy did not improve from 0.78354
1172/1172 [==============================] - 2s 2ms/step - loss: 0.3187 - binary_accuracy: 0.8408 - precision: 0.8558 - recall: 0.8197 - val_loss: 0.7472 - val_binary_accuracy: 0.5023 - val_precision: 0.9167 - val_recall: 0.0053
Epoch 38/1000
1165/1172 [============================>.] - ETA: 0s - loss

1172/1172 [==============================] - 2s 2ms/step - loss: 0.3057 - binary_accuracy: 0.8437 - precision: 0.8591 - recall: 0.8222 - val_loss: 0.4216 - val_binary_accuracy: 0.8548 - val_precision: 1.0000 - val_recall: 0.7097
Epoch 54/1000
1130/1172 [===========================>..] - ETA: 0s - loss: 0.3112 - binary_accuracy: 0.8443 - precision: 0.8588 - recall: 0.8236
Epoch 00054: val_binary_accuracy did not improve from 0.85481
1172/1172 [==============================] - 2s 2ms/step - loss: 0.3109 - binary_accuracy: 0.8447 - precision: 0.8598 - recall: 0.8238 - val_loss: 0.6814 - val_binary_accuracy: 0.6009 - val_precision: 1.0000 - val_recall: 0.2020
Epoch 55/1000
1138/1172 [============================>.] - ETA: 0s - loss: 0.3052 - binary_accuracy: 0.8463 - precision: 0.8598 - recall: 0.8276
Epoch 00055: val_binary_accuracy did not improve from 0.85481
1172/1172 [==============================] - 2s 2ms/step - loss: 0.3052 - binary_accuracy: 0.8466 - precision: 0.8601 - recall: 

1172/1172 [==============================] - 2s 1ms/step - loss: 0.3075 - binary_accuracy: 0.8467 - precision: 0.8650 - recall: 0.8215 - val_loss: 0.6147 - val_binary_accuracy: 0.6724 - val_precision: 1.0000 - val_recall: 0.3450
Epoch 91/1000
1145/1172 [============================>.] - ETA: 0s - loss: 0.3072 - binary_accuracy: 0.8423 - precision: 0.8582 - recall: 0.8202
Epoch 00091: val_binary_accuracy did not improve from 0.85481
1172/1172 [==============================] - 2s 1ms/step - loss: 0.3066 - binary_accuracy: 0.8427 - precision: 0.8584 - recall: 0.8208 - val_loss: 0.4488 - val_binary_accuracy: 0.8390 - val_precision: 1.0000 - val_recall: 0.6780
Epoch 92/1000
1168/1172 [============================>.] - ETA: 0s - loss: 0.3010 - binary_accuracy: 0.8492 - precision: 0.8606 - recall: 0.8333
Epoch 00092: val_binary_accuracy did not improve from 0.85481
1172/1172 [==============================] - 2s 2ms/step - loss: 0.3010 - binary_accuracy: 0.8491 - precision: 0.8607 - recall: 

Epoch 109/1000
1139/1172 [============================>.] - ETA: 0s - loss: 0.3012 - binary_accuracy: 0.8479 - precision: 0.8597 - recall: 0.8318
Epoch 00109: val_binary_accuracy did not improve from 0.85481
1172/1172 [==============================] - 2s 2ms/step - loss: 0.3018 - binary_accuracy: 0.8475 - precision: 0.8592 - recall: 0.8312 - val_loss: 0.5993 - val_binary_accuracy: 0.6763 - val_precision: 1.0000 - val_recall: 0.3527
Epoch 110/1000
1155/1172 [============================>.] - ETA: 0s - loss: 0.3051 - binary_accuracy: 0.8468 - precision: 0.8602 - recall: 0.8282
Epoch 00110: val_binary_accuracy did not improve from 0.85481
1172/1172 [==============================] - 2s 2ms/step - loss: 0.3052 - binary_accuracy: 0.8472 - precision: 0.8606 - recall: 0.8285 - val_loss: 0.4735 - val_binary_accuracy: 0.8147 - val_precision: 1.0000 - val_recall: 0.6296
Epoch 111/1000
1142/1172 [============================>.] - ETA: 0s - loss: 0.3002 - binary_accuracy: 0.8489 - precision: 0.86

...

Epoch 00221: early stopping
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\quekh\Desktop\temp\f1_do3.0_32lr_32lr_32lr_bn_32lr_32lr_32lr_bn_do3.0_Adam_1e04\assets
Epoch 1/1000
1159/1172 [============================>.] - ETA: 0s - loss: 0.5165 - binary_accuracy: 0.7559 - precision_1: 0.8234 - recall_1: 0.6515
Epoch 00001: val_binary_accuracy improved from -inf to 0.49916, saving model to C:\Users\quekh\Desktop\temp\f2_do3.0_32lr_32lr_32lr_bn_32lr_32lr_32lr_bn_do3.0_Adam_1e04\ckpt\weights.01-0.90
1172/1172 [==============================] - 3s 2ms/step - loss: 0.5157 - binary_accuracy: 0.7563 - precision_1: 0.8242 - recall_1: 0.6516 - val_loss: 0.8964 - val_binary_accuracy: 0.4992 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/1000
1159/1172 [

1134/1172 [============================>.] - ETA: 0s - loss: 0.2957 - binary_accuracy: 0.8569 - precision_1: 0.8794 - recall_1: 0.8274
Epoch 00033: val_binary_accuracy did not improve from 0.66091
1172/1172 [==============================] - 2s 2ms/step - loss: 0.2952 - binary_accuracy: 0.8570 - precision_1: 0.8790 - recall_1: 0.8280 - val_loss: 0.7543 - val_binary_accuracy: 0.5462 - val_precision_1: 0.9406 - val_recall_1: 0.0988
Epoch 34/1000
1156/1172 [============================>.] - ETA: 0s - loss: 0.2916 - binary_accuracy: 0.8552 - precision_1: 0.8778 - recall_1: 0.8252
Epoch 00034: val_binary_accuracy did not improve from 0.66091
1172/1172 [==============================] - 2s 2ms/step - loss: 0.2918 - binary_accuracy: 0.8552 - precision_1: 0.8778 - recall_1: 0.8253 - val_loss: 0.8467 - val_binary_accuracy: 0.5097 - val_precision_1: 0.8154 - val_recall_1: 0.0254
Epoch 35/1000
1169/1172 [============================>.] - ETA: 0s - loss: 0.2972 - binary_accuracy: 0.8539 - precisio

Epoch 68/1000
1138/1172 [============================>.] - ETA: 0s - loss: 0.2890 - binary_accuracy: 0.8557 - precision_1: 0.8760 - recall_1: 0.8286
Epoch 00068: val_binary_accuracy did not improve from 0.78834
1172/1172 [==============================] - 2s 2ms/step - loss: 0.2885 - binary_accuracy: 0.8561 - precision_1: 0.8766 - recall_1: 0.8288 - val_loss: 0.6648 - val_binary_accuracy: 0.6287 - val_precision_1: 0.9981 - val_recall_1: 0.2582
Epoch 69/1000
1133/1172 [============================>.] - ETA: 0s - loss: 0.2846 - binary_accuracy: 0.8567 - precision_1: 0.8780 - recall_1: 0.8284
Epoch 00069: val_binary_accuracy improved from 0.78834 to 0.80298, saving model to C:\Users\quekh\Desktop\temp\f2_do3.0_32lr_32lr_32lr_bn_32lr_32lr_32lr_bn_do3.0_Adam_1e04\ckpt\weights.69-0.46
1172/1172 [==============================] - 2s 2ms/step - loss: 0.2842 - binary_accuracy: 0.8568 - precision_1: 0.8784 - recall_1: 0.8283 - val_loss: 0.4585 - val_binary_accuracy: 0.8030 - val_precision_1: 0.9

1149/1172 [============================>.] - ETA: 0s - loss: 0.2809 - binary_accuracy: 0.8641 - precision_1: 0.8824 - recall_1: 0.8406
Epoch 00103: val_binary_accuracy did not improve from 0.85865
1172/1172 [==============================] - 2s 2ms/step - loss: 0.2808 - binary_accuracy: 0.8642 - precision_1: 0.8821 - recall_1: 0.8408 - val_loss: 0.3997 - val_binary_accuracy: 0.8452 - val_precision_1: 0.9993 - val_recall_1: 0.6910
Epoch 104/1000
1163/1172 [============================>.] - ETA: 0s - loss: 0.2797 - binary_accuracy: 0.8636 - precision_1: 0.8807 - recall_1: 0.8407
Epoch 00104: val_binary_accuracy did not improve from 0.85865
1172/1172 [==============================] - 2s 2ms/step - loss: 0.2800 - binary_accuracy: 0.8635 - precision_1: 0.8810 - recall_1: 0.8405 - val_loss: 0.5064 - val_binary_accuracy: 0.7540 - val_precision_1: 0.9991 - val_recall_1: 0.5086
Epoch 105/1000
1144/1172 [============================>.] - ETA: 0s - loss: 0.2851 - binary_accuracy: 0.8588 - precis

KeyboardInterrupt: 

In [ ]:
#Plotting Metrics
if to_plot:
    nth_fold = 0
    nn_list[nth_fold].plot_metrics(0, 20)